https://www.analyticsvidhya.com/blog/2018/10/predicting-stock-price-machine-learningnd-deep-learning-techniques-python/
https://www.researchgate.net/publication/259240183_A_Machine_Learning_Model_for_Stock_Market_Prediction
https://towardsdatascience.com/machine-learning-techniques-applied-to-stock-price-prediction-6c1994da8001
https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/

In [1]:
from fastai.tabular import *
import os
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn import svm, tree
import xgboost
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler


from sklearn import model_selection
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
from sklearn.preprocessing import MinMaxScaler

from datetime import datetime



from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

Using TensorFlow backend.


In [2]:
import glob

In [ ]:
path = Path('/Users/bbabu/fastai/DataSamples')

fnames = sorted(glob.glob('/Users/bbabu/fastai/DataSamples/NSE-*.csv'))
df = pd.DataFrame()

for fname in fnames:
    data = pd.read_csv(fname, header=0)
    print(len(data))
    df = df.append(data)

df.to_csv(path/'NSE.csv', header=True, index=False)

#df[240:249]





In [ ]:
df = pd.read_csv(path/'NSE.csv', header=0)
# df = pd.read_csv(path/'NSE2.csv', header=0)
# df = pd.read_csv(path/'Test.csv', header=0)


# df.dtypes
# df.head()
# len(df)
# df[240:249]

In [ ]:
%matplotlib inline


rcParams['figure.figsize'] = 20,10

scaler = MinMaxScaler(feature_range=(0, 1))


In [ ]:
#setting index as date
df['Date'] = pd.to_datetime(df.Date,format='%Y-%m-%d')
# df['Date'] = pd.to_datetime(df.Date,format='%d-%b-%Y')
df.index = df['Date']

#plot
plt.figure(figsize=(16,8))
plt.plot(df['Close'], label='Close Price history')

In [ ]:
# #sorting
# data = df.sort_index(ascending=True, axis=0)

# data.head()

# #creating a separate dataset
# new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])

# for i in range(0,len(data)):
#     new_data['Date'][i] = data['Date'][i]
#     new_data['Close'][i] = data['Close'][i]

In [ ]:
# #create features
# add_datepart(new_data, 'Date')
# new_data.drop('Elapsed', axis=1, inplace=True)  #elapsed will be the time stamp

In [ ]:
# new_data['mon_fri'] = 0
# for i in range(0,len(new_data)):
#     if (new_data['Dayofweek'][i] == 0 or new_data['Dayofweek'][i] == 4):
#         new_data['mon_fri'][i] = 1
#     else:
#         new_data['mon_fri'][i] = 0

In [ ]:
# new_data.head()



In [ ]:
# #split into train and validation
# train = new_data[:2000]
# valid = new_data[2000:]

# x_train = train.drop('Close', axis=1)
# y_train = train['Close']
# x_valid = valid.drop('Close', axis=1)
# y_valid = valid['Close']

# #implement linear regression
# from sklearn.linear_model import LinearRegression
# model = LinearRegression()
# model.fit(x_train,y_train)

In [ ]:
# #make predictions and find the rmse
# preds = model.predict(x_valid)
# rms=np.sqrt(np.mean(np.power((np.array(y_valid)-np.array(preds)),2)))
# rms

In [ ]:
# #plot
# valid['Predictions'] = 0
# valid['Predictions'] = preds

# valid.index = new_data[2000:].index
# train.index = new_data[:2000].index

# plt.plot(train['Close'])
# plt.plot(valid[['Close', 'Predictions']])

In [ ]:
#creating dataframe
data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
# new_data = pd.DataFrame(index=range(0,test,columns=['Date', 'Close'])


for i in range(0,len(data)):
# for i in range(0,test):
    new_data['Date'][i] = data['Date'][i]
    new_data['Close'][i] = data['Close'][i]

#setting index
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

#creating train and test sets
dataset = new_data.values

train = dataset[0:2000,:]
valid = dataset[2000:,:]

#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(120,len(train)):
    x_train.append(scaled_data[i-120:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=128))
model.add(Dense(1))
model.add(Dropout(1)) 


model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=200, batch_size=4, verbose=2)

#predicting 246 values, using past 120 from the train data
inputs = new_data[len(new_data) - len(valid) - 120:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(120,inputs.shape[0]):
    X_test.append(inputs[i-120:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)


In [ ]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
rms

In [ ]:
#for plotting
train = new_data[:2000]
valid = new_data[2000:]
valid['Predictions'] = closing_price
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])

print(valid)


In [ ]:
# ??dropout()